In [1]:
import SimpleITK as sitk
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import zoom
import ipywidgets as widgets
from IPython.display import display
import os
from collections import defaultdict
from tqdm import tqdm  
import random
from tqdm.notebook import tqdm

Inital File Set Orgnaizaiton

In [2]:
# Dictionary to store file paths organized by patient name/MRN
patient_files = defaultdict(lambda: defaultdict(list))

directories = ["D:\\CTH_archive\\TMAX_DICOM", "D:\\CTH_archive\\CTH_DICOM", "D:\\CTH_archive\\CTP_DICOM"]
directory_names = ["TMAX_DICOM", "CTH_DICOM", "CTP_DICOM"]

for directory, dir_name in zip(directories, directory_names):
    print(f"Processing directory: {directory}")
    if os.path.exists(directory):
        for root, dirs, files in os.walk(directory):
            for file in files:
                file_path = os.path.join(root, file)
                # Split the path to extract the patient name
                parts = root.split("\\")
                # Find the index of the main directory to ensure the patient name is correctly extracted
                try:
                    index = parts.index(dir_name) + 1
                    patient_name = parts[index]
                    patient_files[patient_name][dir_name].append(file_path)
                except (IndexError, ValueError) as e:
                    print(f"Error processing {file_path}: {e}")
    else:
        print(f"Error: Directory {directory} does not exist")

# Filter patients with files in all three directories
patients_with_all_folders = [
    patient for patient, folders in patient_files.items()
    if set(folders.keys()) == set(directory_names)
]

print("Number of patients with all folders:", len(patients_with_all_folders))

# Print the organized file paths for patients with files in all three folders

#for patient in patients_with_all_folders:
#    print(f"Patient: {patient}")
#    for folder, file_paths in patient_files[patient].items():
#        print(f"  {folder}:")
#        for file_path in file_paths:
#            print(f"    {file_path}")
#    print()

# Dictionary to store file paths organized by patient name/MRN
patient_files = defaultdict(lambda: defaultdict(list))

directories = ["D:\\CTH_archive\\TMAX_DICOM", "D:\\CTH_archive\\CTH_DICOM", "D:\\CTH_archive\\CTP_DICOM"]
directory_names = ["TMAX_DICOM", "CTH_DICOM", "CTP_DICOM"]



for directory, dir_name in zip(directories, directory_names):
    if os.path.exists(directory):
        for root, dirs, files in os.walk(directory):
            for file in files:
                file_path = os.path.join(root, file)
                # Split the path to extract the patient name
                parts = root.split("\\")
                # Find the index of the main directory to ensure the patient name is correctly extracted
                try:
                    index = parts.index(dir_name) + 1
                    patient_name = parts[index]
                    patient_files[patient_name][dir_name].append(file_path)
                except (IndexError, ValueError) as e:
                    print(f"Error processing {file_path}: {e}")
    else:
        print(f"Error: Directory {directory} does not exist")

# Filter patients with files in all three directories and store in a new variable
filtered_patient_files = {
    patient: folders for patient, folders in patient_files.items()
    if set(folders.keys()) == set(directory_names)
}

Processing directory: D:\CTH_archive\TMAX_DICOM
Processing directory: D:\CTH_archive\CTH_DICOM
Processing directory: D:\CTH_archive\CTP_DICOM
Number of patients with all folders: 85


In [3]:
def read_dicom_series(directory):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(directory)
    reader.SetFileNames(dicom_names)
    image = reader.Execute()

    if image.GetNumberOfComponentsPerPixel() > 1:
        channels = [sitk.VectorIndexSelectionCast(image, i) for i in range(image.GetNumberOfComponentsPerPixel())]
        image = sum(channels) / len(channels)

    image = sitk.Cast(image, sitk.sitkFloat32)
    return image


In [4]:
def resize_image(image, new_size=(512, 512), fill_value=0):
    original_size = image.GetSize()
    original_spacing = image.GetSpacing()
    original_origin = image.GetOrigin()
    original_direction = image.GetDirection()

    # Maintain the original depth while resizing the X and Y dimensions
    new_size_3D = (new_size[0], new_size[1], original_size[2])

    # Calculate the new spacing for the X and Y dimensions while maintaining the original Z spacing
    new_spacing = [(original_size[0] * original_spacing[0]) / new_size[0],
                   (original_size[1] * original_spacing[1]) / new_size[1],
                   original_spacing[2]]

    # Resample the image to the new spacing, using the new size for the X and Y dimensions
    resample_filter = sitk.ResampleImageFilter()
    resample_filter.SetOutputSpacing(new_spacing)
    resample_filter.SetSize(new_size_3D)
    resample_filter.SetOutputPixelType(image.GetPixelID())
    resample_filter.SetInterpolator(sitk.sitkLinear)
    resample_filter.SetDefaultPixelValue(fill_value)
    
    # Use an identity transform to maintain the position of the image in space
    identity_transform = sitk.Transform(3, sitk.sitkIdentity)
    resample_filter.SetTransform(identity_transform)

    resampled_image = resample_filter.Execute(image)

    # Set the original origin and direction to the resampled image to preserve spatial information
    resampled_image.SetOrigin(original_origin)
    resampled_image.SetDirection(original_direction)

    return resampled_image


In [5]:
transforms_dir = r'D:\CTH_archive\Transforms'  # Directory to save the transforms
if not os.path.exists(transforms_dir):
    os.makedirs(transforms_dir)

resampled_images = {}  # To store the resampled images for each patient
resampled_images_transform = {}  # To store the transform files for each patient

def remove_background(image):
    threshold_value = -300  # Adjust based on your CT scans
    binary_image = sitk.BinaryThreshold(image, lowerThreshold=threshold_value, upperThreshold=3000, insideValue=1, outsideValue=0)
    morph_radius = [2, 2, 2]  # Specify as a list or tuple
    binary_image = sitk.BinaryMorphologicalOpening(binary_image, morph_radius)
    cc_filter = sitk.ConnectedComponentImageFilter()
    cc_image = cc_filter.Execute(binary_image)
    stats_filter = sitk.LabelIntensityStatisticsImageFilter()
    stats_filter.Execute(cc_image, binary_image)
    largest_label = max(stats_filter.GetLabels(), key=lambda x: stats_filter.GetPhysicalSize(x))
    binary_image = sitk.BinaryThreshold(cc_image, lowerThreshold=largest_label, upperThreshold=largest_label, insideValue=1, outsideValue=0)
    
    # Cast the binary_image to the same pixel type as the original image
    casted_binary_image = sitk.Cast(binary_image, image.GetPixelID())

    return image * casted_binary_image

for patient in tqdm(patients_with_all_folders, desc='Processing Patients'):
    if 'CTP_DICOM' in patient_files[patient] and 'CTH_DICOM' in patient_files[patient]:
        ctp_dicom_directory = os.path.dirname(patient_files[patient]['CTP_DICOM'][0])
        cth_dicom_directory = os.path.dirname(patient_files[patient]['CTH_DICOM'][0])

        # Read DICOM series
        moving_image = read_dicom_series(ctp_dicom_directory)
        fixed_image = read_dicom_series(cth_dicom_directory)
        
        # Resize the fixed image to 512x512
        #fixed_image = resize_image(fixed_image)

        # Remove background from the moving and fixed images
        segmented_moving_image = remove_background(moving_image)
        segmented_fixed_image = remove_background(fixed_image)
        
    
           # Read DICOM series and apply histogram normalization
        moving_image = read_dicom_series(ctp_dicom_directory)
        moving_image_min = float(sitk.GetArrayFromImage(moving_image).min())
        moving_image_max = float(sitk.GetArrayFromImage(moving_image).max())
        moving_image = sitk.IntensityWindowing(moving_image,
                                            windowMinimum=moving_image_min,
                                            windowMaximum=moving_image_max,
                                            outputMinimum=0.0,
                                            outputMaximum=1.0)

        fixed_image = read_dicom_series(cth_dicom_directory)
        fixed_image_min = float(sitk.GetArrayFromImage(fixed_image).min())
        fixed_image_max = float(sitk.GetArrayFromImage(fixed_image).max())
        fixed_image = sitk.IntensityWindowing(fixed_image,
                                            windowMinimum=fixed_image_min,
                                            windowMaximum=fixed_image_max,
                                            outputMinimum=0.0,
                                            outputMaximum=1.0)


        #fig, axs = plt.subplots(2, 2, figsize=(10, 8))
        #axs[0, 0].imshow(sitk.GetArrayFromImage(fixed_image)[fixed_image.GetSize()[2] // 2], cmap='gray')
        #axs[0, 0].set_title('Original Fixed Image')
        #axs[0, 0].axis('off')
        #axs[0, 1].imshow(sitk.GetArrayFromImage(segmented_fixed_image)[fixed_image.GetSize()[2] // 2], cmap='gray')
        #axs[0, 1].set_title('Segmented Fixed Image')
        #axs[0, 1].axis('off')
        #axs[1, 0].imshow(sitk.GetArrayFromImage(moving_image)[moving_image.GetSize()[2] // 2], cmap='gray')
        #axs[1, 0].set_title('Original Moving Image')
        #axs[1, 0].axis('off')
        #axs[1, 1].imshow(sitk.GetArrayFromImage(segmented_moving_image)[moving_image.GetSize()[2] // 2], cmap='gray')
        #axs[1, 1].set_title('Segmented Moving Image')
        #axs[1, 1].axis('off')
        #plt.show()

        registration_method = sitk.ImageRegistrationMethod()
        registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
        registration_method.SetMetricSamplingPercentage(0.6, sitk.sitkWallClock)
        registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
        registration_method.SetOptimizerAsGradientDescentLineSearch(learningRate=0.5, numberOfIterations=300, convergenceMinimumValue=1e-6, convergenceWindowSize=20)
        registration_method.SetOptimizerScalesFromPhysicalShift()
        registration_method.SetShrinkFactorsPerLevel(shrinkFactors=[8, 4, 2])
        registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas=[4, 2, 1])
        registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()
        initial_transform = sitk.CenteredTransformInitializer(fixed_image, moving_image, sitk.AffineTransform(fixed_image.GetDimension()), sitk.CenteredTransformInitializerFilter.MOMENTS)
        registration_method.SetInitialTransform(initial_transform)
        try:
            final_transform = registration_method.Execute(segmented_fixed_image, segmented_moving_image)
            resampled_image = sitk.Resample(segmented_moving_image, segmented_fixed_image, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())
            resampled_images[patient] = resampled_image
            transform_file = os.path.join(transforms_dir, f'{patient}_transform.h5')
            sitk.WriteTransform(final_transform, transform_file)
            resampled_images_transform[patient] = transform_file
            print(f"Registration successful for patient: {patient}. Transform saved to {transform_file}")
        except RuntimeError as e:
            print(f"Registration failed for patient {patient}: {e}")



Processing Patients:   0%|          | 0/85 [00:00<?, ?it/s]

Registration successful for patient: ALFORD_BARBARA 4024996. Transform saved to D:\CTH_archive\Transforms\ALFORD_BARBARA 4024996_transform.h5
Registration successful for patient: ALLAH_MAJUSTICE 2621774. Transform saved to D:\CTH_archive\Transforms\ALLAH_MAJUSTICE 2621774_transform.h5
Registration successful for patient: BATTLE_MARIA 8399298. Transform saved to D:\CTH_archive\Transforms\BATTLE_MARIA 8399298_transform.h5
Registration successful for patient: BOGER_DAVID_S 2532249. Transform saved to D:\CTH_archive\Transforms\BOGER_DAVID_S 2532249_transform.h5


In [ ]:
def display_image_slices(fixed_image, moving_image, transformed_image):
    # Convert SimpleITK images to arrays for easier manipulation
    fixed_image_array = sitk.GetArrayFromImage(fixed_image)
    transformed_image_array = sitk.GetArrayFromImage(transformed_image)

    # Determine the maximum number of slices from the fixed and transformed images
    max_slices = max(fixed_image.GetSize()[2], transformed_image.GetSize()[2])

    def update_slice(slice_idx):
        # Create figure with 2 subplots
        fig, axs = plt.subplots(1, 2, figsize=(10, 5))

        # Display the fixed image slice
        axs[0].imshow(fixed_image_array[slice_idx], cmap='gray')
        axs[0].set_title('Fixed Image')

        # Display the overlay: fixed image slice with the transformed image slice overlaid with transparency
        if slice_idx < transformed_image_array.shape[0]:
            axs[1].imshow(fixed_image_array[slice_idx], cmap='gray')
            axs[1].imshow(transformed_image_array[slice_idx], cmap='jet', alpha=0.5)  # Adjust alpha for desired transparency
            axs[1].set_title('Overlay: Fixed + Transformed Image')
        else:
            axs[1].text(0.5, 0.5, 'Slice not available', horizontalalignment='center', verticalalignment='center')
            axs[1].set_title('Overlay: Fixed + Transformed Image')

        # Turn off axis for all subplots
        #for ax in axs:
        #    ax.axis('off')

        plt.show()

    # Create a slider widget for slice selection
    slice_slider = widgets.IntSlider(min=0, max=max_slices-1, step=1, value=max_slices//2, description='Slice')

    # Display the widget and use `interactive_output` to connect the slider with the update function
    interactive_output = widgets.interactive_output(update_slice, {'slice_idx': slice_slider})
    display(slice_slider, interactive_output)

In [ ]:
patient_id = "BATTLE_MARIA 8399298"

ctp_dicom_directory = os.path.dirname(patient_files[patient_id]['CTP_DICOM'][0])
cth_dicom_directory = os.path.dirname(patient_files[patient_id]['CTH_DICOM'][0])

moving_image = read_dicom_series(ctp_dicom_directory)
fixed_image = read_dicom_series(cth_dicom_directory)

# Getting the transformed (resampled) image from your results
transformed_image = resampled_images[patient_id]

display_image_slices(fixed_image, moving_image, transformed_image)

In [ ]:
def display_fusion(fixed_image, registered_image):
    fixed_array = sitk.GetArrayFromImage(fixed_image)
    registered_array = sitk.GetArrayFromImage(registered_image)

    # Ensure the images are in the same size for overlay
    registered_resampled = sitk.Resample(registered_image, fixed_image)

    # Convert SimpleITK images to arrays
    fixed_array = sitk.GetArrayFromImage(fixed_image)
    registered_array = sitk.GetArrayFromImage(registered_resampled)

    # Choose a slice in the middle of the volume to display
    slice_idx = fixed_array.shape[0] // 2

    # Create a fusion image by overlaying the registered image on the fixed image
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(fixed_array[slice_idx], cmap='gray')
    plt.title('Fixed Image')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(fixed_array[slice_idx], cmap='gray')
    plt.imshow(registered_array[slice_idx], cmap='jet', alpha=0.5)  # Adjust alpha for transparency
    plt.title('Fusion Visualization')
    plt.axis('off')

    plt.show()

In [ ]:
for patient_id in patient_files.keys():
    # Check if 'CTP_DICOM' and 'CTH_DICOM' lists have elements
    if patient_files[patient_id]['CTP_DICOM'] and patient_files[patient_id]['CTH_DICOM']:
        # Define directories
        ctp_dicom_directory = os.path.dirname(patient_files[patient_id]['CTP_DICOM'][0])
        cth_dicom_directory = os.path.dirname(patient_files[patient_id]['CTH_DICOM'][0])

        # Read DICOM series
        moving_image = read_dicom_series(ctp_dicom_directory)
        fixed_image = read_dicom_series(cth_dicom_directory)

        # Assuming transformations have been applied and the transformed images are stored
        transformed_image = resampled_images.get(patient_id)  # Using .get() to avoid KeyError if patient_id is not in resampled_images

        if transformed_image:  # Check if transformed_image exists for the patient_id
            # Display fusion for the middle slice of each patient's fixed and transformed images
            display_fusion(fixed_image, transformed_image)
            pass
        else:
            print(f"No transformed image for patient {patient_id}")
    else:
        print(f"Missing DICOM files for patient {patient_id}")


In [ ]:
def convert_series_to_nifti(input_directory, output_file):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(input_directory)
    reader.SetFileNames(dicom_names)
    image_series = reader.Execute()
    
    # Convert to numpy array to manipulate the pixel data directly
    img_array = sitk.GetArrayFromImage(image_series)

    # Check if the image needs to be converted to grayscale
    if image_series.GetNumberOfComponentsPerPixel() > 1:
        img_array = np.mean(img_array, axis=-1).astype(np.uint16)

        img_array[:, :35, :] = 0  # Remove TAMX label
        img_array[:, :, :35] = 0 # Remove scale on  the left
    
    # Convert the numpy array back to a SimpleITK Image
    processed_image = sitk.GetImageFromArray(img_array)
    processed_image.SetSpacing(image_series.GetSpacing())
    processed_image.SetOrigin(image_series.GetOrigin())
    processed_image.SetDirection(image_series.GetDirection())

    # Write the processed image as a NIfTI file
    sitk.WriteImage(processed_image, output_file)

root_directory = r'D:\CTH_archive\TMAX_DICOM'

for patient_dir in os.listdir(root_directory):
    patient_path = os.path.join(root_directory, patient_dir)
    if os.path.isdir(patient_path):
        for series_dir in os.listdir(patient_path):
            series_path = os.path.join(patient_path, series_dir)
            if os.path.isdir(series_path):
                output_nifti_file = os.path.join(patient_path, f"{series_dir}.nii")
                print(f"Converting {series_path} to NIfTI...")
                convert_series_to_nifti(series_path, output_nifti_file)

In [ ]:
#New map registraion

def display_image_slices(fixed_image, resampled_image):
    fixed_image_array = sitk.GetArrayFromImage(fixed_image)
    resampled_image_array = sitk.GetArrayFromImage(resampled_image)

    max_slices = max(fixed_image.GetSize()[2], resampled_image.GetSize()[2])

def display_image_slices(fixed_image, moving_image, transformed_image):
    fixed_image_array = sitk.GetArrayFromImage(fixed_image)
    moving_image_array = sitk.GetArrayFromImage(moving_image)  # Original moving image
    transformed_image_array = sitk.GetArrayFromImage(transformed_image)  # Transformed (registered) moving image

    max_slices = max(fixed_image_array.shape[0], transformed_image_array.shape[0])

    def update_slice(slice_idx):
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))

        axs[0].imshow(fixed_image_array[slice_idx, :, :], cmap='gray')
        axs[0].set_title('Fixed Image')

        if slice_idx < moving_image_array.shape[0]:
            axs[1].imshow(moving_image_array[slice_idx, :, :], cmap='gray')
            axs[1].set_title('Original Moving Image')
        else:
            axs[1].text(0.5, 0.5, 'Slice not available', horizontalalignment='center', verticalalignment='center')
            axs[1].set_title('Original Moving Image')

        if slice_idx < transformed_image_array.shape[0]:
            axs[2].imshow(fixed_image_array[slice_idx, :, :], cmap='gray')
            axs[2].imshow(transformed_image_array[slice_idx, :, :], cmap='jet', alpha=0.5)
            axs[2].set_title('Overlay: Fixed + Transformed Image')
        else:
            axs[2].text(0.5, 0.5, 'Slice not available', horizontalalignment='center', verticalalignment='center')
            axs[2].set_title('Overlay: Fixed + Transformed Image')

        for ax in axs:
            ax.axis('off')
        plt.show()

    slice_slider = widgets.IntSlider(min=0, max=max_slices-1, step=1, value=max_slices//2, description='Slice')
    interactive_output = widgets.interactive_output(update_slice, {'slice_idx': slice_slider})
    display(slice_slider, interactive_output)


output_dir = os.path.join(r'D:\CTH_archive\Registered_NIfTI')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

successful_patients = list(resampled_images.keys())

for patient_id in successful_patients:
    patient_directory = os.path.join(r'D:\CTH_archive\TMAX_DICOM', patient_id)
    
    nifti_files = [f for f in os.listdir(patient_directory) if f.endswith('.nii')]
    if not nifti_files:
        print(f"No NIfTI file found for patient {patient_id}.")
        continue

    nifti_file = nifti_files[0]
    nifti_file_path = os.path.join(patient_directory, nifti_file)
    moving_image = sitk.ReadImage(nifti_file_path)
    
    ctp_directory_base = os.path.join(r'D:\CTH_archive\CTP_DICOM', patient_id)
    study_dirs = [d for d in os.listdir(ctp_directory_base) if os.path.isdir(os.path.join(ctp_directory_base, d))]
    if not study_dirs:
        print(f"No CTP study found for patient {patient_id} in {ctp_directory_base}.")
        continue

    ctp_directory = os.path.join(ctp_directory_base, study_dirs[0])
    ctp_image = read_dicom_series(ctp_directory)

    fixed_image_base_dir = os.path.join(r'D:\CTH_archive\CTH_DICOM', patient_id)
    
    series_dirs = [d for d in os.listdir(fixed_image_base_dir) if os.path.isdir(os.path.join(fixed_image_base_dir, d))]
    if not series_dirs:
        print(f"No DICOM series found for patient {patient_id} in {fixed_image_base_dir}.")
        continue

    fixed_image_dir = os.path.join(fixed_image_base_dir, series_dirs[0])
    fixed_image = read_dicom_series(fixed_image_dir)
     
    print(f"CTP image size: {ctp_image.GetSize()}")
    print(f"Moving image size: {moving_image.GetSize()}")
    print(f"CTH image size: {fixed_image.GetSize()}")

    print(f'CTP image spacing: {ctp_image.GetSpacing()}')
    print(f'Moving image spacing: {moving_image.GetSpacing()}')
    print(f'CTH image spacing: {fixed_image.GetSpacing()}')

    print(f'CTP image origin: {ctp_image.GetOrigin()}')
    print(f'Moving image origin: {moving_image.GetOrigin()}')
    print(f'CTH image origin: {fixed_image.GetOrigin()}')

    print(f'CTP image direction: {ctp_image.GetDirection()}')   
    print(f'Moving image direction: {moving_image.GetDirection()}')
    print(f'CTH image direction: {fixed_image.GetDirection()}')


    # Resample the moving image to match the fixed image using your preferred method
    desired_size = [fixed_image.GetSize()[0], fixed_image.GetSize()[1], ctp_image.GetSize()[2]]
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(moving_image)
    resampler.SetSize(desired_size)
    resampler.SetOutputSpacing([moving_image.GetSpacing()[i] * (moving_image.GetSize()[i] / desired_size[i]) for i in range(3)])
    resampler.SetTransform(sitk.Transform())
    resampler.SetInterpolator(sitk.sitkLinear)
    resized_moving_image = resampler.Execute(moving_image)
    resized_moving_image.SetSpacing(ctp_image.GetSpacing())
    resized_moving_image.SetOrigin(ctp_image.GetOrigin())
    resized_moving_image.SetDirection(ctp_image.GetDirection())

    # Load the final transform from the file path stored in resampled_images
    transform_file_path = resampled_images_transform[patient_id]
    if not os.path.exists(transform_file_path):
        print(f"Transform file for patient {patient_id} not found at {transform_file_path}.")
        continue
    final_transform = sitk.ReadTransform(transform_file_path)

    # Apply the final transform to the resized moving image
    resampled_image = sitk.Resample(resized_moving_image, 
                                    fixed_image, 
                                    final_transform, 
                                    sitk.sitkLinear, 
                                    0.0, 
                                    fixed_image.GetPixelID())

    print(f"Resampled image size: {resampled_image.GetSize()}")
    print(f"Resampled image spacing: {resampled_image.GetSpacing()}")
    print(f"Resampled image origin: {resampled_image.GetOrigin()}") 

    # Save the resampled image
    output_path = os.path.join(output_dir, f'{patient_id}_registered.nii')
    sitk.WriteImage(resampled_image, output_path)
    print(f'output_path: {output_path}')
    display_image_slices(fixed_image, moving_image, resampled_image)
    print(f"Processed and saved registered image for patient: {patient_id}")



In [ ]:
import os
import SimpleITK as sitk

def read_dicom_series(directory):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(directory)
    reader.SetFileNames(dicom_names)
    image = reader.Execute()
    return image

output_dir = os.path.join(r'D:\CTH_archive\Registered_NIfTI')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

successful_patients = list(resampled_images.keys())

for patient_id in successful_patients:
    patient_directory = os.path.join(r'D:\CTH_archive\TMAX_DICOM', patient_id)
    nifti_files = [f for f in os.listdir(patient_directory) if f.endswith('.nii')]
    
    if not nifti_files:
        print(f"No NIfTI file found for patient {patient_id}.")
        continue

    nifti_file = nifti_files[0]
    nifti_file_path = os.path.join(patient_directory, nifti_file)
    moving_image = sitk.ReadImage(nifti_file_path)
    
    ctp_directory_base = os.path.join(r'D:\CTH_archive\CTP_DICOM', patient_id)
    study_dirs = [d for d in os.listdir(ctp_directory_base) if os.path.isdir(os.path.join(ctp_directory_base, d))]
    
    if not study_dirs:
        print(f"No CTP study found for patient {patient_id} in {ctp_directory_base}.")
        continue

    ctp_directory = os.path.join(ctp_directory_base, study_dirs[0])
    ctp_image = read_dicom_series(ctp_directory)

    fixed_image_base_dir = os.path.join(r'D:\CTH_archive\CTH_DICOM', patient_id)
    series_dirs = [d for d in os.listdir(fixed_image_base_dir) if os.path.isdir(os.path.join(fixed_image_base_dir, d))]
    
    if not series_dirs:
        print(f"No DICOM series found for patient {patient_id} in {fixed_image_base_dir}.")
        continue

    fixed_image_dir = os.path.join(fixed_image_base_dir, series_dirs[0])
    fixed_image = read_dicom_series(fixed_image_dir)

    print(f"patient_id: {patient_id}************************************")
    print(f"CTP image size: {ctp_image.GetSize()}, number of slices: {ctp_image.GetDepth()}")
    print(f"Moving image size: {moving_image.GetSize()}, number of slices: {moving_image.GetDepth()}")
    print(f"CTH image size: {fixed_image.GetSize()}, number of slices: {fixed_image.GetDepth()}")

    print(f'CTP image spacing: {ctp_image.GetSpacing()}')
    print(f'Moving image spacing: {moving_image.GetSpacing()}')
    print(f'CTH image spacing: {fixed_image.GetSpacing()}')

    print(f'CTP image origin: {ctp_image.GetOrigin()}')
    print(f'Moving image origin: {moving_image.GetOrigin()}')
    print(f'CTH image origin: {fixed_image.GetOrigin()}')

    print(f'CTP image direction: {ctp_image.GetDirection()}')   
    print(f'Moving image direction: {moving_image.GetDirection()}')
    print(f'CTH image direction: {fixed_image.GetDirection()}')

    # Add here additional metadata printouts if relevant, using fixed_image, ctp_image or moving_image
    # For example: print(f'CTH Patient Position: {fixed_image.GetMetaData("0018|5100")}')

    # Resample the moving image to match the fixed image using your preferred method
    desired_size = [fixed_image.GetSize()[0], fixed_image.GetSize()[1], moving_image.GetSize()[2]]
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(moving_image)
    resampler.SetSize(desired_size)
    resampler.SetOutputSpacing([moving_image.GetSpacing()[i] * (moving_image.GetSize()[i] / desired_size[i]) for i in range(3)])
    resampler.SetTransform(sitk.Transform())
    resampler.SetInterpolator(sitk.sitkLinear)
    resized_moving_image = resampler.Execute(moving_image)
    resized_moving_image.SetSpacing(ctp_image.GetSpacing())
    resized_moving_image.SetOrigin(ctp_image.GetOrigin())
    resized_moving_image.SetDirection(ctp_image.GetDirection())

    print(f"Resampled image size: {resized_moving_image.GetSize()}")
    print(f"Resampled image spacing: {resized_moving_image.GetSpacing()}")
    print(f"Resampled image origin: {resized_moving_image.GetOrigin()}")
    print(f"Resampled image direction: {resized_moving_image.GetDirection()}")


In [ ]:
import os
import numpy as np
import pydicom
from scipy.ndimage import zoom

def resize_and_pad(array, target_size=(512, 512), order=1):
    """Resize a 2D array to the target size and pad with zeros to match the target size exactly."""
    zoom_factors = np.array(target_size) / np.array(array.shape)
    resized_array = zoom(array, zoom_factors, order=order)
    pad_width = [(0, 0)] * array.ndim  # Initialize pad width for each dimension
    for i, dim in enumerate(resized_array.shape):
        pad = (target_size[i] - dim) / 2
        pad_width[i] = (int(np.floor(pad)), int(np.ceil(pad)))
    padded_array = np.pad(resized_array, pad_width, mode='constant', constant_values=0)
    return padded_array.astype(array.dtype)

def process_series(series_path, dest_root):
    """Process a DICOM series by resizing and padding all slices, then save them in the new folder structure."""
    for file in os.listdir(series_path):
        if file.endswith(".dcm"):
            dicom_path = os.path.join(series_path, file)
            ds = pydicom.dcmread(dicom_path)

            if ds.pixel_array.ndim != 2:
                print(f"Skipping {dicom_path}: Expected a 2D array, got {ds.pixel_array.ndim}D array instead.")
                continue

            resized_padded_img = resize_and_pad(ds.pixel_array)
            ds.PixelData = resized_padded_img.tobytes()
            ds.Rows, ds.Columns = resized_padded_img.shape

            # Extract components from the source path
            components = series_path.split(os.sep)[-2:]  # Last two components should be Patient Name MRN and Date Time
            patient_info = components[0]
            date_time = components[1].replace(" ", "_")  # Replace spaces with underscores for consistency

            # Construct the destination path
            dest_path = os.path.join(dest_root, patient_info, date_time)

            if not os.path.exists(dest_path):
                os.makedirs(dest_path)

            ds.save_as(os.path.join(dest_path, file))

def process_base_folder(base_folder, dest_root):
    """Process all series in the base folder."""
    for root, dirs, files in os.walk(base_folder):
        if any(file.endswith('.dcm') for file in files):
            process_series(root, dest_root)

base_folder = "D:/CTH_archive/CTH_DICOM"
dest_root = "D:/CTH_archive/CTH_DICOM_RESIZE"
process_base_folder(base_folder, dest_root)
